# Short Circuit Example

This notebook contains an example of a short circuit calculation using `power-grid-model`.
The following points are covered
 * Construction of the model
 * Run short circuit calculation, and its relevant function arguments

This notebook skips most of the common features across all calculations of power-grid-model like updating model, batch calculations and error handling.

It serves as an example of how to use the Python API. For detailed API documentation, refer to
[Python API reference](https://power-grid-model.readthedocs.io/en/stable/api_reference/python-api-reference.html)
and [Native Data Interface](https://power-grid-model.readthedocs.io/en/stable/advanced_documentation/native-data-interface.html).


## Example Network

We use a simple network with 3 nodes, 1 source, 3 lines, and 2 loads. As shown below:

```
 -----------------------line_8---------------
 |                                          |
node_1 ---line_3--- node_2 ----line_5---- node_6 ---- fault_11
 |                    |                     |
source_10          sym_load_4           sym_load_7
```

The 3 nodes are connected in a triangular way by 3 lines.

In [1]:
# some basic imports
import numpy as np
import pandas as pd

from power_grid_model import LoadGenType
from power_grid_model import PowerGridModel, CalculationMethod, CalculationType, FaultType, FaultPhase
from power_grid_model import initialize_array

## Input Dataset

Please refer to [Components](https://power-grid-model.readthedocs.io/en/stable/user_manual/components.html) for detailed explanation of all component types and their input/output attributes.

In [2]:
# node
node = initialize_array("input", "node", 3)
node["id"] = np.array([1, 2, 6])
node["u_rated"] = [10.5e3, 10.5e3, 10.5e3]

# line
line = initialize_array("input", "line", 3)
line["id"] = [3, 5, 8]
line["from_node"] = [1, 2, 1]
line["to_node"] = [2, 6, 6]
line["from_status"] = [1, 1, 1]
line["to_status"] = [1, 1, 1]
line["r1"] = [0.25, 0.25, 0.25]
line["x1"] = [0.2, 0.2, 0.2]
line["c1"] = [10e-6, 10e-6, 10e-6]
line["tan1"] = [0.0, 0.0, 0.0]
line["i_n"] = [1000, 1000, 1000]

# load
sym_load = initialize_array("input", "sym_load", 2)
sym_load["id"] = [4, 7]
sym_load["node"] = [2, 6]
sym_load["status"] = [1, 1]
sym_load["type"] = [LoadGenType.const_power, LoadGenType.const_power]
sym_load["p_specified"] = [20e6, 10e6]
sym_load["q_specified"] = [5e6, 2e6]

# source
source = initialize_array("input", "source", 1)
source["id"] = [10]
source["node"] = [1]
source["status"] = [1]
source["u_ref"] = [1.0]

# source
fault = initialize_array("input", "fault", 1)
fault["id"] = [11]
fault["status"] = [1]
fault["fault_object"] = [6]
fault["fault_type"] = [FaultType.three_phase]
fault["fault_phase"] = [FaultPhase.abc]
fault["r_f"] = [0.1]
fault["x_f"] = [0.1]

# all
input_data = {
    "node": node,
    "line": line,
    "sym_load": sym_load,
    "source": source,
    "fault": fault
}


**Note: During a single calculation, all types of faults should be similar.**

## Validation (optional)
For efficiency reasons, most of the data is not explicitly validated in the power grid model. However, in most cases, a power flow calculation will fail/crash if the data is invalid. Often with a low level error message that is hard to relate to the original objects. Therfore, it is recommended to always validate your data before constructing a PowerGridModel instance.

The simplest and most effective way to validate your data is by using `assert_valid_input_data()` which will throw an error if it encounters any invalid data. See [Validation Examples](https://github.com/PowerGridModel/power-grid-model/blob/main/docs/examples/Validation%20Examples.ipynb) for more detailed information on the validation functions.

In [3]:
from power_grid_model.validation import assert_valid_input_data
assert_valid_input_data(input_data=input_data, calculation_type=CalculationType.short_circuit)

## Construction

The construction of the model is just calling the constructor of `PowerGridModel`.


In [4]:
model = PowerGridModel(input_data)

## One-time Short circuit Calculation

You can call the method `calculate_short_circuit` to do a one-time calculation based on the current network data in the model.

The short circuit calculation has the following settings as arguments along with its defaults:
 * calculation_method: CalculationMethod.iec60909,
 * voltage_scaling_factor_c: 1.1

Currently, there is only one calculation method for short-circuit which calculates as per IEC 60909. The `voltage_scaling_factor_c` is scaling of voltage source based on operational conditions.


In [5]:
output_data = model.calculate_short_circuit(calculation_method=CalculationMethod.iec60909)
output_data

{'node': array([(1, 1, [1.07539348, 1.07539348, 1.07539348], [6519.22650954, 6519.22650954, 6519.22650954], [-0.02079111, -2.11518622,  2.07360399]),
        (2, 1, [0.75227027, 0.75227027, 0.75227027], [4560.39613786, 4560.39613786, 4560.39613786], [-0.00230211, -2.09669721,  2.09209299]),
        (6, 1, [0.42934657, 0.42934657, 0.42934657], [2602.77527185, 2602.77527185, 2602.77527185], [ 0.04539925, -2.04899585,  2.13979435])],
       dtype={'names': ['id', 'energized', 'u_pu', 'u', 'u_angle'], 'formats': ['<i4', 'i1', ('<f8', (3,)), ('<f8', (3,)), ('<f8', (3,))], 'offsets': [0, 4, 8, 32, 56], 'itemsize': 80, 'aligned': True}),
 'line': array([(3, 1, [ 6119.72533835,  6119.72533835,  6119.72533835], [-0.7372701 , -2.8316652 ,  1.357125  ], [ 6131.26794632,  6131.26794632,  6131.26794632], [ 2.40219628,  0.30780118, -1.78659392]),
        (5, 1, [ 6131.26794632,  6131.26794632,  6131.26794632], [-0.73939637, -2.83379147,  1.35499873], [ 6138.97747082,  6138.97747082,  6138.97747082],

### Result Dataset

We can also print a result dataset of node and line by converting the array to dataframe.

In [6]:
print("------fault result------")
display(pd.DataFrame(output_data["fault"].tolist(), columns=output_data["fault"].dtype.names))
print("------node result------")
display(pd.DataFrame(output_data["node"].tolist(), columns=output_data["node"].dtype.names))
print("------line result------")
display(pd.DataFrame(output_data["line"].tolist(), columns=output_data["line"].dtype.names))

------fault result------


,id,energized,i_f,i_f_angle
0,11,1,"[18404.400446310916, 18404.400446310916, 18404...","[-0.7399989108285706, -2.834394013221766, 1.35..."


------node result------


,id,energized,u_pu,u,u_angle
0,1,1,"[1.0753934800545326, 1.0753934800545326, 1.075...","[6519.226509539654, 6519.226509539654, 6519.22...","[-0.02079111382761733, -2.115186216220813, 2.0..."
1,2,1,"[0.7522702679435408, 0.7522702679435408, 0.752...","[4560.396137855829, 4560.396137855829, 4560.39...","[-0.0023021117177247433, -2.0966972141109204, ..."
2,6,1,"[0.4293465725267879, 0.4293465725267878, 0.429...","[2602.7752718518336, 2602.775271851833, 2602.7...","[0.04539925256887769, -2.0489958498243177, 2.1..."


------line result------


,id,energized,i_from,i_from_angle,i_to,i_to_angle
0,3,1,"[6119.725338347851, 6119.725338347851, 6119.72...","[-0.737270099772722, -2.8316652021659174, 1.35...","[6131.267946321706, 6131.267946321706, 6131.26...","[2.4021962831505785, 0.30780118075738305, -1.7..."
1,5,1,"[6131.267946321705, 6131.267946321705, 6131.26...","[-0.7393963704392148, -2.8337914728324103, 1.3...","[6138.977470816719, 6138.977470816719, 6138.97...","[2.400861093608147, 0.30646599121495144, -1.78..."
2,8,1,"[12255.7967133573, 12255.7967133573, 12255.796...","[-0.738767428467694, -2.8331625308608896, 1.35...","[12265.425447771773, 12265.425447771773, 12265...","[2.4019604415245324, 0.30756533913133705, -1.7..."


## Batch Calculations

The batch calculations are mentioned in detail in the [Power Flow Example](https://github.com/PowerGridModel/power-grid-model/tree/main/docs/examples/Power%20Flow%20Example.ipynb). Short circuit batch calculations are carried out in similar way.